# Pipeline for extracting data for Inpainting
## By Doncey Albin

In [2]:
import os
import numpy as np
import open3d as o3d
import osmnx as ox
import pickle

# Internal
from tools.labels import labels
from tools.convert_oxts_pose import *
import tools.osm_building as osm_building
from tools.utils import *

## 1) Load poses.

In [3]:
oxts_pose_file_path = "/Users/donceykong/Desktop/kitti360Scripts/data/2013_05_28_drive_0005_sync_pose2oxts.txt"
xyz_point_clouds, xyz_positions = get_pointcloud_from_txt(oxts_pose_file_path) # Create point clouds from XYZ positions

## 2) Load pointcloud with "building" and "unlabeled" points.

In [4]:
ply_file_path = '/Users/donceykong/Desktop/kitti360Scripts/data/output3D.ply'
point_cloud_3D = o3d.io.read_point_cloud(ply_file_path)

# Get the points as a numpy array
points_3D = np.asarray(point_cloud_3D.points)

# Set all z-components to zero to create a 2D point cloud
points_2D = points_3D.copy()
points_2D[:, 2] = 0

# Create a new 2D point cloud from the modified points
point_cloud_2D = o3d.geometry.PointCloud()
point_cloud_2D.points = o3d.utility.Vector3dVector(points_2D)

## 3) Filter buildings within bounds near pose path. This makes cycling through to see if edges were hit much faster.

In [5]:
osm_file_path = '/Users/donceykong/Desktop/kitti360Scripts/data/map_0005.osm'

# Filter features for buildings
building_features = ox.features_from_xml(osm_file_path, tags={'building': True})
print(f"\nlen(buildings): {len(building_features)}")

threshold_dist = 0.0008 
building_list, building_line_set = get_buildings_near_poses(building_features, xyz_positions, threshold_dist)


len(buildings): 4507


## 4) Create edge_points_list via discretizing by a set number.

In [6]:
num_points_per_edge = 100
discretize_all_building_edges(building_list, num_points_per_edge)


## 5) Filter lidar points that are a +/- radius distance away from and of the edge points in the edge_points list.

### TODO: Maybe here would be a good point to do some sort of scan-matching so that the buildings and OSM-polygons are better aligned

In [7]:
radius = 0.000008
calc_points_on_building_edges(building_list, point_cloud_3D, point_cloud_2D, radius)

Building: 1 / 715
Building: 2 / 715
Building: 3 / 715
Building: 4 / 715
Building: 5 / 715
Building: 6 / 715
Building: 7 / 715
Building: 8 / 715
Building: 9 / 715
Building: 10 / 715
Building: 11 / 715
Building: 12 / 715
Building: 13 / 715
Building: 14 / 715
Building: 15 / 715
Building: 16 / 715
Building: 17 / 715
Building: 18 / 715
Building: 19 / 715
Building: 20 / 715
Building: 21 / 715
Building: 22 / 715
Building: 23 / 715
Building: 24 / 715
Building: 25 / 715
Building: 26 / 715
Building: 27 / 715
Building: 28 / 715
Building: 29 / 715
Building: 30 / 715
Building: 31 / 715
Building: 32 / 715
Building: 33 / 715
Building: 34 / 715
Building: 35 / 715
Building: 36 / 715
Building: 37 / 715
Building: 38 / 715
Building: 39 / 715
Building: 40 / 715
Building: 41 / 715
Building: 42 / 715
Building: 43 / 715
Building: 44 / 715
Building: 45 / 715
Building: 46 / 715
Building: 47 / 715
Building: 48 / 715
Building: 49 / 715
Building: 50 / 715
Building: 51 / 715
Building: 52 / 715
Building: 53 / 715
Bu

## 6) Filter buildings with no internal points.

In [8]:
hit_building_list, hit_building_line_set = get_building_hit_list(building_list)

# # Save hit_building_list to a file
# with open('hit_building_list_seq_0005.pkl', 'wb') as file:
#     pickle.dump(hit_building_list, file)

## 7) Save filtered building points.

In [9]:
filtered_points = [point for building in hit_building_list for point in building.points]
filtered_point_cloud = o3d.geometry.PointCloud()
filtered_point_cloud.points = o3d.utility.Vector3dVector(filtered_points)
filtered_point_cloud.paint_uniform_color([1, 0, 0])  # Blue color for buildings

PointCloud with 188063 points.

## Optional: View bounds on building edges which points were included

In [10]:
all_edge_circles = get_building_edge_bounds(hit_building_list, radius)
all_edge_circles.paint_uniform_color([0, 1, 0])

PointCloud with 10371000 points.

## 8) Visualize the filtered point cloud

In [12]:
from tools.plotly_viz import *
# Visualize building points on OG point cloud
# o3d.visualization.draw_geometries([point_cloud_3D, filtered_point_cloud, hit_building_line_set])

# Visualize buildings that have been hit by laser and the clouds
o3d.visualization.draw_geometries([all_edge_circles, filtered_point_cloud, hit_building_line_set])

# # Use plotly to display in-line
# o3d.visualization.draw_geometries = draw_geometries # replace function
# o3d.visualization.draw_geometries([point_cloud_3D])
# o3d.visualization.draw_geometries([filtered_point_cloud])
# o3d.visualization.draw_geometries([hit_building_line_set])